### Load dataset

In [ ]:
import numpy as np
import pickle as pkl
from typing import List
from dataset_objects.bbox import BoundingBox
import os

SHAPE_FILE = "data/shape.pkl"
if os.path.exists(SHAPE_FILE):
    with open(SHAPE_FILE, 'rb') as h:
        shape_array = pkl.load(h)

[cells_width, cells_height, G, N, F, dataset_size] = shape_array

region_centers = np.zeros(
    shape=(
        cells_width,
        cells_height,
        G
    ),
    dtype=np.float64
)

normalized_region_centers = np.zeros(
    shape=(
        cells_width,
        cells_height,
        G
    ),
    dtype=np.float64
)

first_input_features = np.zeros(
    shape=(
        dataset_size,
        N,
        F
    ),
    dtype=np.float64
)

second_input_features = np.zeros(
    shape=(
        dataset_size,
        2
    ),
    dtype=np.float64
)

outputs = np.zeros(
    shape=(dataset_size),
    dtype=np.float64
)

cells:List[List[BoundingBox]]

In [ ]:
import os
import pickle as pkl

FOLDER = "data/"
INPUTS_1_FILE = FOLDER + "inputs_1.pkl"
INPUTS_2_FILE = FOLDER + "inputs_2.pkl"
OUTPUTS_FILE = FOLDER + "outputs.pkl"
CENTERS_FILE = FOLDER + "centers.pkl"
N_CENTERS_FILE = FOLDER + "normal_centers.pkl"
REGIONS_FILE = FOLDER + "regions.pkl"

if os.path.exists(INPUTS_1_FILE):
    with open(INPUTS_1_FILE, 'rb') as h:
        first_input_features = pkl.load(h)

if os.path.exists(INPUTS_2_FILE):
    with open(INPUTS_2_FILE, 'rb') as h:
        second_input_features = pkl.load(h)

if os.path.exists(OUTPUTS_FILE):
    with open(OUTPUTS_FILE, 'rb') as h:
        outputs = pkl.load(h)

if os.path.exists(CENTERS_FILE):
    with open(CENTERS_FILE, 'rb') as h:
        region_centers = pkl.load(h)

if os.path.exists(N_CENTERS_FILE):
    with open(N_CENTERS_FILE, 'rb') as h:
        normalized_region_centers = pkl.load(h)

if os.path.exists(REGIONS_FILE):
    with open(REGIONS_FILE, 'rb') as h:
        cells = pkl.load(h)

In [ ]:
PERCENT_USED_FOR_TRAINING = 0.8

training_total = round(PERCENT_USED_FOR_TRAINING * outputs.size)

training_x1 = first_input_features[:training_total]
training_x2 = second_input_features[:training_total]
training_y = outputs[:training_total]

testing_x1 = first_input_features[training_total:]
testing_x2 = second_input_features[training_total:]
testing_y = outputs[training_total:]

### Model parameters

In [ ]:
LSTM_layers = 4                     # LSTM Layer count
dense_layers = 3                    # Dense layer count
EPOCHS = 5                          # Total epochs
PERCENT_USED_OR_VALIDATION = 0.15   # Validation %
BATCH_SIZE = 64                     # Batch size

### Model building

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

lstm_input = layers.Input(shape=np.shape(first_input_features[0]))
dense_input = layers.Input(shape=np.shape(second_input_features[0]))
lstm = layers.LSTM(LSTM_layers)(lstm_input)
concatenated = layers.concatenate([lstm, dense_input])
dense = layers.Dense(dense_layers, activation="relu")(concatenated)
dense = layers.Dense(1, activation="sigmoid")(dense)
final_model = keras.Model([lstm_input, dense_input], dense)

print(final_model.summary())

### Train

In [ ]:
final_model.compile(
    optimizer=keras.optimizers.SGD(),
    loss='mean_absolute_error',
    metrics=[
        tf.keras.metrics.MeanSquaredError(name="Mean Squared Error"),
        tf.keras.metrics.MeanAbsoluteError(name="Mean Absolute Error"),
        tf.keras.metrics.LogCoshError(name="Log Cosh"),
        tf.keras.metrics.RootMeanSquaredError(name="Root Mean Squared Error")
    ]
)

fit_history = final_model.fit(
    x=[training_x1, training_x2],
    y=training_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    callbacks=None,
    validation_split=PERCENT_USED_OR_VALIDATION,
    validation_data=None,
    shuffle='batch',
    class_weight=None,
    sample_weight=None,
    initial_epoch=0,
    steps_per_epoch=None,
    validation_steps=None,
    validation_batch_size=None,
    validation_freq=1,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
)

### Test

In [ ]:
print(final_model.metrics)
final_model.evaluate(
    x=[testing_x1, testing_x2],
    y=testing_y,
    batch_size=BATCH_SIZE,
    verbose=1,
    sample_weight=None,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False,
    return_dict=False,
)

In [ ]:
VERSION_NAME = "okers"
FILENAME = VERSION_NAME + '.h5'
final_model.save(FILENAME)

In [ ]:
outbound_regions = []
for x in range(cells_width):
    for y in range(cells_height):
        outbound_regions.append(
            {
                "regionId": "",
                "predictor": "",
                "center": {
                    "latitude": region_centers[x][y][0],
                    "longitude": region_centers[x][y][1]
                },
                "normalizedCenter": {
                    "latitude": normalized_region_centers[x][y][0],
                    "longitude": normalized_region_centers[x][y][1]
                },
                "risk": -1,
                "bounds": {
                    "coordinates": [
                        [
                            cells[x][y].south_west.latitude,
                            cells[x][y].south_west.longitude
                        ],
                        [
                            cells[x][y].south_east.latitude,
                            cells[x][y].south_east.longitude
                        ],
                        [
                            cells[x][y].north_east.latitude,
                            cells[x][y].north_east.longitude
                        ],
                        [
                            cells[x][y].north_west.latitude,
                            cells[x][y].north_west.longitude
                        ],
                    ],
                    "type": "Polygon"
                }
            }
        )

In [ ]:
import requests
import json
with open(FILENAME, 'rb') as f:
    requests.post(
        url='http://localhost:8080/prediction/model/persist',
        data={"bounds": json.dumps(outbound_regions)},
        files=dict(predictiveModel = f)
    )

In [ ]:
X = dataset_size - 9218
result: tf.Tensor = final_model([
    np.reshape(first_input_features[X], (1, 7, 4)),
    np.reshape(second_input_features[X], (1, 2))
])
print(result)
print(outputs[X])